# Thermal

gdsfactory has an FEM plugin that you can use for thermal simulations.

In [ ]:
import gmsh
import gdsfactory as gf
from gdsfactory.simulation.gmsh.mesh2D import mesh2D
from gdsfactory.simulation.thermal import solve_thermal

gf.tech.LAYER_STACK.layers["heater"].thickness = 0.13
gf.tech.LAYER_STACK.layers["heater"].zmin = 2.2

heater1 = gf.components.straight_heater_metal(length=50, heater_width=2)
heater2 = gf.components.straight_heater_metal(length=50, heater_width=2).move([0, -10])

heaters = gf.Component("heaters")
heaters << heater1
heaters

In [ ]:
print(gf.tech.LAYER_STACK.layers.keys())

In [ ]:
geometry = mesh2D(
    heaters,
    ((25, -25), (25, 25)),
    base_resolution=0.4,
    exclude_layers=((1, 10),),
    padding=(10, 10, 1, 1),
    refine_resolution={(1, 0): 0.01, (47, 0): 0.005},
)

gmsh.write("mesh.msh")
gmsh.clear()
geometry.__exit__()

solve_thermal(
    mesh_filename="mesh.msh",
    thermal_conductivity={"(47, 0)": 28, "oxide": 1.38, "(1, 0)": 148},
    specific_conductivity={"(47, 0)_0": 2.3e6},
    thermal_diffusivity={
        "(47, 0)": 28 / 598 / 5240,
        "oxide": 1.38 / 709 / 2203,
        "(1, 0)": 148 / 711 / 2330,
    },
    # specific_heat={"(47, 0)_0": 598, 'oxide': 709, '(1, 0)': 711},
    # density={"(47, 0)_0": 5240, 'oxide': 2203, '(1, 0)': 2330},
    currents={"(47, 0)_0": 0.007},
)

![heater](https://i.imgur.com/TU6wKqf.png)


![time](https://i.imgur.com/uaR6klM.png)